In [8]:
# Hi, I did not run it, but hopefully it can be a nice way - at least a large step approximation - to the final implementation
#enjoy

#from networkx.algorithms.threshold import weights_to_creation_sequence
import re
import torch
import srt
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

#my global truth:)
checkpoint = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(checkpoint)
#translation is a seq2seq problem
model = M2M100ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer.src_lang = "cs"
tokenizer.tgt_lang = "en"
forced_bos_token_id = tokenizer.lang_code_to_id["en"]

def load_srt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        subtitles = list(srt.parse(file.read()))
    return subtitles

def preprocess_text(subtitles):
    cleaned_subs = {}
    for sub in subtitles:
        text = sub.content.strip()
        text = re.sub(r'\s+', ' ', text)
        cleaned_subs[sub.index] = {"start": sub.start, "end": sub.end, "text": text}  
    return cleaned_subs

def tokenize_texts(text_dict):
    texts = [entry["text"] for entry in text_dict.values()]
    tokenized = tokenizer(texts, return_tensors="pt", truncation=True)

    max_len = max(len(seq) for seq in tokenized["input_ids"])

    # pad
    input_ids = [seq + [tokenizer.pad_token_id] * (max_len - len(seq)) for seq in tokenized["input_ids"]]
    attention_mask = [[1] * len(seq) + [0] * (max_len - len(seq)) for seq in tokenized["input_ids"]]

    tokenized_inputs = {
        "input_ids": torch.tensor(input_ids),
        "attention_mask": torch.tensor(attention_mask),
    }

    return tokenized_inputs

def translate_texts(tokenized_inputs):
    inputs = tokenizer(tokenized_inputs, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            forced_bos_token_id=forced_bos_token_id,
            max_length=200
        )
    translated_texts = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
    return translated_texts

def format_output(subtitles, translated_texts):
    translated_dict = {
        sub_id: {
            "original": subtitles[sub_id]["text"],
            "translated": translated_texts[i],
            "start": subtitles[sub_id]["start"],
            "end": subtitles[sub_id]["end"],
        }
        for i, sub_id in enumerate(subtitles)
    }
    return translated_dict

if __name__ == "__main__":
    import json

    file_path = ""
    subtitles = load_srt_file(file_path)

    cleaned_subs = preprocess_text(subtitles)

    tokenized_inputs = tokenize_texts(cleaned_subs)

    translated_output = format_output(cleaned_subs, translate_texts(tokenized_inputs))
    print(json.dumps(translated_output, indent=2, ensure_ascii=False))
    
    #if we want we still can save them to SRT
    # if we want we can do evaluation metrics with BLEU as
    #n-grams: type of (n piece of) subsequent tokens to calculate BLEU metrics
    #class transformers.modeling_outputs.Seq2SeqModelOutput

KeyboardInterrupt: 